<a href="https://colab.research.google.com/github/anuthereaper/PythonLibrary/blob/main/Table_storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install azure-data-tables

In [ ]:
pip install names

In [ ]:
from azure.core.credentials import AzureNamedKeyCredential
from azure.data.tables import TableServiceClient

#credential = AzureNamedKeyCredential("my_account_name", "my_access_key")
#table_service_client = TableServiceClient(endpoint="https://<my_account_name>.table.core.windows.net", credential=credential)

connection_string = "xxxxxxxxxxxxxxxx"
tablename = "mytable"

table_service_client = TableServiceClient.from_connection_string(conn_str=connection_string)
table_client = table_service_client.get_table_client(table_name=tablename)

In [ ]:
#Insert 1 row to table storage
from datetime import datetime

my_entity = {
            "PartitionKey": "color",
            "RowKey": "brand",
            "text": "Marker",
            "color": "Purple",
            "price": 4.99,
            "last_updated": datetime.today(),
            "product_id": 1234,
            "inventory_count": 42,
            "barcode": b"135aefg8oj0ld58"
        }
entity = table_client.create_entity(entity=my_entity)

In [ ]:
#BULK UPLOAD 1 by 1 using the latest datatables SDK.
from azure.data.tables import TableServiceClient
from datetime import datetime
from random import randrange
import random
import names
import json
from datetime import timedelta
import uuid

Number_of_docs = 50
d1 = datetime.strptime('1/1/2008 1:30 PM', '%m/%d/%Y %I:%M %p')
d2 = datetime.strptime('1/1/2009 4:50 AM', '%m/%d/%Y %I:%M %p')

def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

def create_entity(id):
    my_entity = {"PartitionKey" : str(uuid.uuid4()),
                "RowKey" : str(id),
                "employee_id" : str(id),
                "first_name" : names.get_first_name(),
                "last_name" : names.get_last_name(),
                "regn_no" : "TEST00000000"+ str(id),
                "start_date" : random_date(d1,d2),
                "salary" : random.randint(12000, 2000000),
                "inserted_at" : str(datetime.now())
                }
    return my_entity

starttime = datetime.utcnow()
print("Starting ingestion: ", starttime.strftime("%Y-%m-%d %H:%M:%S.%f"))
for i in range(Number_of_docs):
    my_entity = create_entity(i)
    entity = table_client.create_entity(entity=my_entity)
    if (i+1)%100 == 0 or (i == Number_of_docs-1):
        print(str(i+1) + " documents completed")

endtime = datetime.utcnow()
print("\nrun_sample done :" + endtime.strftime("%Y-%m-%d %H:%M:%S.%f"))
print("Time taken :" + str(endtime-starttime))
print("Number of messages :" + str(Number_of_docs))

In [ ]:
#BULK UPLOAD USING LATEST TABLE STORAGE API with batching (https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/tables/azure-data-tables/samples)
from azure.data.tables import TableClient
from datetime import datetime
from random import randrange
import random
import names
import json
from datetime import timedelta
import uuid

conn='xxxxxxxxxxxxxx'
tablename='mytable'      
table_client = TableClient.from_connection_string(conn_str=conn,table_name=tablename)
Number_of_docs = 1000
d1 = datetime.strptime('1/1/2008 1:30 PM', '%m/%d/%Y %I:%M %p')
d2 = datetime.strptime('1/1/2009 4:50 AM', '%m/%d/%Y %I:%M %p')

def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

def create_entity(id):
    my_entity = {"PartitionKey" : "PARTITION1",
                "RowKey" : str(id),
                "employee_id" : str(id),
                "first_name" : names.get_first_name(),
                "last_name" : names.get_last_name(),
                "regn_no" : "TEST00000000"+ str(id),
                "start_date" : random_date(d1,d2),
                "salary" : random.randint(12000, 2000000),
                "inserted_at" : str(datetime.now())
                }
    return my_entity

starttime = datetime.utcnow()
print("Starting ingestion: ", starttime.strftime("%Y-%m-%d %H:%M:%S.%f"))
batch_no = 0
operations = []
for i in range(Number_of_docs):
    i = i + 1
    batch_no = batch_no + 1
    my_entity = create_entity(i)
    operations.append(("upsert", my_entity))
    if (batch_no > 50):
        table_client.submit_transaction(operations)
        batch_no = 0
        operations = []
    else:      
        batch_no = batch_no + 1
    
if (batch_no > 0):
    table_client.submit_transaction(operations)

endtime = datetime.utcnow()
print("\nrun_sample done :" + endtime.strftime("%Y-%m-%d %H:%M:%S.%f"))
print("Time taken :" + str(endtime-starttime))
print("Number of messages :" + str(Number_of_docs))

In [ ]:
#SINGLE QUERY TABLE STORAGE 
#(https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/tables/azure-data-tables/samples)
from azure.data.tables import TableClient

conn='xxxxxxxxxxxxxxxxxx'
tablename='mytable'      
table_client = TableClient.from_connection_string(conn_str=conn,table_name=tablename)

starttime = datetime.utcnow()
parameters = {u"firstname": u"Vikki"}
name_filter = u"first_name eq @firstname"
queried_entities = table_client.query_entities(query_filter=name_filter, select=[u"last_name", u"salary"], parameters=parameters)

for entity_chosen in queried_entities:
    print(entity_chosen)

endtime = datetime.utcnow()
print("\nrun_sample done :" + endtime.strftime("%Y-%m-%d %H:%M:%S.%f"))
print("Time taken :" + str(endtime-starttime))

In [ ]:
#LOAD TEST QUERYING MULTIPLE ENTITIES FROM TABLE STORAGE
#(https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/tables/azure-data-tables/samples)
from azure.data.tables import TableClient
from datetime import datetime
import json
from datetime import timedelta

conn='DefaultEndpointsProtocol=xxxxxxxxxxxxxxxxxxx=;EndpointSuffix=core.windows.net'
tablename='xxxxxxxxxx'      
table_client = TableClient.from_connection_string(conn_str=conn,table_name=tablename)
Number_of_docs = 100


def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

def create_entity(id):
    my_entity = {"PartitionKey" : "PARTITION1",
                "RowKey" : str(id),
                "employee_id" : str(id),
                "first_name" : names.get_first_name(),
                "last_name" : names.get_last_name(),
                "regn_no" : "TEST00000000"+ str(id),
                "start_date" : random_date(d1,d2),
                "salary" : random.randint(12000, 2000000),
                "inserted_at" : str(datetime.now())
                }
    return my_entity

starttime = datetime.utcnow()
print("Starting ingestion: ", starttime.strftime("%Y-%m-%d %H:%M:%S.%f"))

for i in range(Number_of_docs):
    parameters = {u"employeeid": str(i)}
    name_filter = u"employee_id eq @employeeid"
    queried_entities = table_client.query_entities(query_filter=name_filter, select=[u"first_name", "last_name", "salary"], parameters=parameters)
    for entity_chosen in queried_entities:
        print(entity_chosen)

endtime = datetime.utcnow()
print("\nrun_sample done :" + endtime.strftime("%Y-%m-%d %H:%M:%S.%f"))
print("Total time taken :" + str(endtime-starttime))
print("Number of messages :" + str(Number_of_docs))
print("Avg time for each query :" + str((endtime-starttime)/Number_of_docs))

In [ ]:
# DELETE A SINGLE ENTITY FROM TABLE STORAGE
#(https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/tables/azure-data-tables/samples)
from azure.data.tables import TableClient
from datetime import datetime
import json
from datetime import timedelta

conn='xxxxxxxxxxxxxxxxxxxxxxxxxx'
tablename='mytable'      
table_client = TableClient.from_connection_string(conn_str=conn,table_name=tablename)
PartitionKey = 'PARTITION1'
RowKey = str(1)

table_client.delete_entity(row_key=RowKey, partition_key=PartitionKey)
print("Successfully deleted!")